In [141]:
import torch
torch.__version__
torch.cuda.is_available()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [142]:
def read_new_email(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    parts = text.split('"')  
    
    subject = parts[1].strip()
    body = parts[3].strip()
    label = parts[5].strip()
    
    new_email = {'Subject':subject, 'Body':body, 'Label':label}
    return new_email

In [143]:
confirmation_emails = []

for i in range(1, 93):
    file_path = f"Data/confirmation/conf{i}.txt"
    new_email = read_new_email(file_path)
    confirmation_emails.append(new_email)

print(len(confirmation_emails))

92


In [144]:
rejection_emails = []

for i in range(1, 105):
    file_path = f"Data/rejections/rejection{i}.txt"
    new_email = read_new_email(file_path)
    rejection_emails.append(new_email)

print(len(rejection_emails))

104


In [145]:
for i in range(0, len(confirmation_emails)):
    subject = confirmation_emails[i]["Subject"]
    body = confirmation_emails[i]["Body"]
    confirmation_emails[i] = f"{subject} {body}"

confirmation_emails[0]

'thanks for your application to spotify! dear applicant, we just got your application for the 2024 summer internship, engineering (new york city) role! even though this is just an automated confirmation email, you should know that we’re truly excited you want to join the band. we’ll get back to you as soon as we can. we get a huge amount of applications, and we look at them all to give everyone fair consideration – so it may take a few weeks (or sometimes months for really popular roles). in the meantime, you can listen to the playlist we’ll play in the office as we read your resume. want to sneak a backstage peek? follow life at spotify on linkedin, instagram, twitter, and youtube. learn more about our culture through our band manifesto, or listen to the greenroom and spoton! podcasts. still curious? check out our hr blog, podcasting website, and engineering blog! all the best, the spotify recruiting team'

In [146]:
for i in range(0, len(rejection_emails)):
    subject = rejection_emails[i]["Subject"]
    body = rejection_emails[i]["Body"]
    rejection_emails[i] = f"{subject} {body}"

rejection_emails[0]

"update on your skydio application - wireless software intern hi applicant, thank you for applying for the wireless software intern role at skydio! we know that there are lots of exciting companies out there, so we appreciate the time you took to apply to ours. after reviewing your application, we have unfortunately decided not to move forward at this time. while we greatly appreciate your interest in skydio, we ultimately decided to proceed with other candidates whose skills and experience align more closely with our needs for this specific role. if you submitted an application to other roles at skydio and haven't heard back, then your resume is still under review and we will be in touch soon with an update. our hiring priorities are constantly evolving, so we encourage you to keep an eye on our jobs page. if you see another role that seems like a good fit, please don't hesitate to apply. in the meantime, we'd like to thank you again for your interest and we wish you all the best in y

In [147]:
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove punctuation
    tokens = [word for word in tokens if word not in string.punctuation]
    
    # Convert to lowercase
    tokens = [word.lower() for word in tokens]
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [148]:
for i in range(0, len(confirmation_emails)):
    new_text = preprocess_text(confirmation_emails[i])
    confirmation_emails[i] = new_text

for i in range(0, len(rejection_emails)):
    new_text = preprocess_text(rejection_emails[i])
    rejection_emails[i] = new_text



In [149]:
confirmation_labels = [1] * len(confirmation_emails)
rejection_labels = [0] * len(rejection_emails)

In [150]:
emails = confirmation_emails + rejection_emails
labels = confirmation_labels + rejection_labels

In [151]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text data to TF-IDF features
vectorizer = TfidfVectorizer(stop_words='english', max_features=338)  # Limit to 338 features
X = vectorizer.fit_transform(emails).toarray()
y = np.array(labels)

In [152]:
import torch
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensors = torch.tensor(X_train, dtype=torch.float32)
X_test_tensors = torch.tensor(X_test, dtype=torch.float32)
y_train_tensors = torch.tensor(y_train, dtype=torch.float32)
y_test_tensors = torch.tensor(y_test, dtype=torch.float32)

In [153]:
#Accuracy Function

def acc_fn(y_true, y_pred):
    y_pred = torch.round(y_pred)
    correct = torch.eq(y_true, y_pred).sum().item()
    accuracy = (correct/len(y_pred)) * 100
    return accuracy

In [269]:
from torch import nn

# class EmailModel(nn.Module):
#     def __init__(self, input_size):
#         super(EmailModel, self).__init__()
#         self.fc1 = nn.Linear(input_size, 512)
#         self.fc2 = nn.Linear(512, 128)
#         self.fc3 = nn.Linear(128, 64)
#         self.fc4 = nn.Linear(64, 1)
#         self.relu = nn.ReLU()
#         self.sigmoid = nn.Sigmoid()
    
#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.relu(self.fc3(x))
    

#         x = self.fc4(x)
#         x = self.sigmoid(x)
#         return x

# input_size = 338
# model_1 = EmailModel(input_size).to(device)
# model_1

class EmailModel(nn.Module):
    def __init__(self, input_size):
        super(EmailModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 1)
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.3)
        self.layer_norm1 = nn.LayerNorm(512)
        self.layer_norm2 = nn.LayerNorm(256)
        self.layer_norm3 = nn.LayerNorm(128)
        self.layer_norm4 = nn.LayerNorm(64)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.leaky_relu(self.layer_norm1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer_norm2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer_norm3(self.fc3(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.layer_norm4(self.fc4(x)))
        x = self.fc5(x)
        x = self.sigmoid(x)
        return x

input_size = 338
model_1 = EmailModel(input_size).to(device)
model_1


EmailModel(
  (fc1): Linear(in_features=338, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=1, bias=True)
  (leaky_relu): LeakyReLU(negative_slope=0.01)
  (dropout): Dropout(p=0.3, inplace=False)
  (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (layer_norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (layer_norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (layer_norm4): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (sigmoid): Sigmoid()
)

In [274]:
from torch import nn, optim

# Define the model
input_size = X_train_tensors.shape[1]
model = EmailModel(input_size).to(device)
X_test_tensors = X_test_tensors.to(device)
X_train_tensors = X_train_tensors.to(device)
y_train_tensors = y_train_tensors.to(device)
y_test_tensors = y_test_tensors.to(device)

# Loss function and optimizer
loss_fn = nn.BCELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Training loop
epochs = 22

for epoch in range(epochs):
    model.train()

    # Forward pass
    train_preds = model(X_train_tensors).squeeze()
    train_loss = loss_fn(train_preds, y_train_tensors)
    train_acc = acc_fn(y_train_tensors, train_preds)

    # Backward pass and optimization
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    # Evaluation
    model.eval()
    with torch.inference_mode():
        test_preds = model(X_test_tensors).squeeze()
        test_loss = loss_fn(test_preds, y_test_tensors)
        test_acc = acc_fn(y_test_tensors, test_preds)
        
    print(f"Epoch: {epoch+1} | Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")

Epoch: 1 | Train Loss: 0.6984, Train Acc: 50.64% | Test Loss: 0.6735, Test Acc: 52.50%
Epoch: 2 | Train Loss: 0.6417, Train Acc: 58.33% | Test Loss: 0.5986, Test Acc: 77.50%
Epoch: 3 | Train Loss: 0.6342, Train Acc: 66.03% | Test Loss: 0.5636, Test Acc: 85.00%
Epoch: 4 | Train Loss: 0.5824, Train Acc: 76.92% | Test Loss: 0.4752, Test Acc: 92.50%
Epoch: 5 | Train Loss: 0.5326, Train Acc: 85.26% | Test Loss: 0.4305, Test Acc: 85.00%
Epoch: 6 | Train Loss: 0.4604, Train Acc: 88.46% | Test Loss: 0.4107, Test Acc: 85.00%
Epoch: 7 | Train Loss: 0.3982, Train Acc: 91.67% | Test Loss: 0.3359, Test Acc: 90.00%
Epoch: 8 | Train Loss: 0.3180, Train Acc: 96.79% | Test Loss: 0.3020, Test Acc: 87.50%
Epoch: 9 | Train Loss: 0.2817, Train Acc: 97.44% | Test Loss: 0.2921, Test Acc: 87.50%
Epoch: 10 | Train Loss: 0.2329, Train Acc: 98.72% | Test Loss: 0.2879, Test Acc: 90.00%
Epoch: 11 | Train Loss: 0.1914, Train Acc: 99.36% | Test Loss: 0.3023, Test Acc: 90.00%
Epoch: 12 | Train Loss: 0.1735, Train Acc

In [275]:
torch.save(model.state_dict(), 'EmailModel_params.pth')
